In [1]:
from json import dumps, loads
import time
import threading 
import ast
import unittest
import requests
# chainsapce
from chainspacecontract import transaction_to_solution
# from chainspacecontract.examples.surge import contract as surge_contract
from chainspacecontract.examples import surge
# crypto
from chainspacecontract.examples.utils import setup, key_gen, pack
from chainspaceapi import ChainspaceClient
G = setup()[0]


In [2]:
global_client = ChainspaceClient(port=5000)


In [3]:
init_transaction = surge.init()
init_token1 = init_transaction['transaction']['outputs'][0]
init_token2 = init_transaction['transaction']['outputs'][1]
init_token3 = init_transaction['transaction']['outputs'][2]
init_token4 = init_transaction['transaction']['outputs'][3]
global_client.process_transaction(init_transaction)

inputs: ()
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": [], "methodID": "init", "parameters": [], "outputs": ["{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {}}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}


<Response [200]>

In [4]:
type(init_token1)

chainspacecontract.contract.ChainspaceObject

In [5]:
r1 = surge.SurgeClient(init_token1) # location=0
r2 = surge.SurgeClient(init_token2) # location=0
rep1 = surge.SREPClient() 

votes = (r1.cast_srep_vote(rep1.pub), r2.cast_srep_vote(rep1.pub))
rep1.create_srep_client(votes) # location=0
# r1.submit_bid('EBBuy', 50)
# r2.submit_bid('EBSell', 50)


inputs: ('{"type": "InitToken", "location": 0}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_616abd97764a2b04f8d9982a44cdcaddfd7b9eddc77c70c147f0d0ea00d67474"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd03dd6649c55b0e70e4753b6404d7d055e4b2ea79a93e3b2b46b579462c", "92c71d002b6b2df5f5f59b72957ec0845be99ae1f138aacb43a609d24d3c424780c71d002b5829c6ac292793e8ad29f6eebba6d2ec0440a8e22c58ada71ceb950f"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03dd6649c55b0e70e4753b6404d7d055e4b2ea79a93e3b2b46b579462c\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03dd6649c55b0e70e4753b6404d7d055e4b2ea79a93e3b2b46b579462c\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03dd6649c55b0e70e4753b6404d7d055e4b2ea79a93e3b2b46b579462c\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surg

In [6]:
t1 = threading.Thread(target=r1.submit_bid, args=('EBBuy',50)) 
t2 = threading.Thread(target=r2.submit_bid, args=('EBSell',50)) 
t3 = threading.Thread(target=rep1.accept_bids) 

In [7]:
t1.start()
t2.start()
t3.start()
t1.join()
t2.join()
t3.join()

inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd03dd6649c55b0e70e4753b6404d7d055e4b2ea79a93e3b2b46b579462c"}',)
reference_inputs: ()
inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd0237048aa2eb433b466fa7aea97bd2c6696bdcf75e4655f0001b0e7cb6"}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
 POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_30990706014e4d7352df724ccc837ab147eb334b546d5aee464352a6d38cadd4"], "methodID": "submit_bid_proof", "parameters": ["EBSell", "92c71d002b8625873115a42948dc214ea1ff5a1312274dec0a60e6723b79ff98d6c71d002bc6fb466ee1bf2f6b3b895967fea428eafeb7bd47e6569155f930420c"], "outputs": ["{\"bid_type\": \"EBSell\", \"type\": \"BidProof\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0237048aa2eb433b466fa7aea97bd2c6696bdcf75e4655f0001b0e7cb6\", \"quantity_sig\": \"92c71d002b85707052f6714b94564d43183d338f9010478de826619c6819cb5d02c71d002b19775b

Traceback (most recent call last):
  File "/home/shinjan/surge/chainspacecontract/chainspacecontract/examples/surge.py", line 671, in accept_bids_checker
    equate(accept_bids['location'], b['location'])
TypeError: 'function' object has no attribute '__getitem__'


HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}


In [8]:
# '{}|{}'.format(50, loads(r1.ebtoken)['pub'])
# global_client.fix_json("{'type':'InitToken','location':0, 'value':{'foo':'bar'}}")
buy_bids = global_client.get_objects({'location':0, 'type':'EBBuy'})
buy_bids = tuple(buy_bids)
loads(buy_bids[0])['location']

# print loads(objs[0])['type']
# bid_proofs = rep1.client.get_objects({ 'type':'EBBuy'})
# print init_token1


http://127.0.0.1:5000/api/1.0/objects?status=1
HTTP/1.1 200 OK


0